# Get started

## Install the Addin

If you have Autodesk Revit open, close it. The addin need to register a dockable panel on startup!

In [ ]:
dotnet build Jowsy.Revit.KernelAddin -c "Debug R24"

Open Revit and go to the "Add-ins" tab where a button "Show Dockable Pane" will let you show the kernel viewer.

## Install VS Code Polyglot Notebook extension

In [ ]:
# 1. Build the project
dotnet build Jowsy.DotNet.Interactive.Extensions

# Clear any older versions of this extension package from your NuGet cache
rm ~/.nuget/packages/Jowsy.DotNet.Interactive.Extensions -Force -Recurse -ErrorAction Ignore

dotnet pack Jowsy.DotNet.Interactive.Extensions -c Debug /p:PackageVersion=1.0.0

$nugets = Get-ChildItem -Recurse Jowsy.DotNet.Interactive.Extensions*.nupkg

$nugetPackage = $nugets[0].FullName

In [ ]:
#!set --value @pwsh:nugetPackage --name nugetPackage
var nugetSource = System.IO.Path.GetDirectoryName(nugetPackage);
nugetSource

In [ ]:
#r "nuget:Jowsy.DotNet.Interactive.Extensions"

#i @csharp:nugetSource

## Connecting to the Revit kernel
We use the *#!connect* magic command to establish connection to the embedded revit kernel. Remember to start the kernel on the Revit side. Goto Addins->Net Interactive->Show Dockable Panel.

In [ ]:
#!connect revit --kernel-name revit24 --revit-version 2024

If connection is established global variables are created for the current UIApplication, UIDocument and Document. Variables are shown in the kernel viewer as seen below. Those variables can be referenced in code cells chosen to run on the Revit kernel.

![](screenshot-variables.png)

In order to get IntelliSense you can import Revit API references. For the moment IntelliSense is not supported when using the revit kernel. A workaround is to uncomment the connector directive to activate the csharp-kernel.

In [ ]:
#r "nuget:Revit.RevitApi.x64, 2023.0.0"
#r "nuget:Revit.RevitApiUi.x64, 2023.0.0"
using Autodesk.Revit.DB;
using Autodesk.Revit.UI;

## Querying the Revit Database
Use display method to output the result from a query. 

NOTE: Objects of type Autodesk.DB.Elements are configured to be rendered as HTML. 

The formatter only recurse to a specific depth (default 3).

In [ ]:
#!revit24
   
   var query = new FilteredElementCollector(doc).OfClass(typeof(Wall))
                                                 .Cast<Wall>();

display(query);


If you just return the variable it will be defined and shown in Variables in the kernel UI in Revit.

In [ ]:
#!revit24
   
   var wall = new FilteredElementCollector(doc).OfClass(typeof(Wall))
                                                 .Cast<Wall>()
                                                 .First();

wall

Now you can reference wall in other cells.

In [ ]:
#!revit24
display(wall.Name);

Level level = doc.GetElement(wall.LevelId) as Level;

display(level.Name);

## Variable Sharing


The .NET Interactive kernel enables you to write code in multiple languages within a single notebook. The revit kernel is a C#-kernel but is run on a different process than the composite kernel. This means that we cannot reference a variable from the revit kernel directly but we need to use *#!set* magic command to send the value from the revit kernel to a another kernel. If you try to share a Autodesk Element object it will try to convert it to json but it will probably fail. Some simple types such as XYZ are serializable.

In the example below we extract the wall profile and draws it live in the notebook using Hypar IO.

**Run the cell below and then pick a wall in Revit!**

In [ ]:
//Adapted from https://thebuildingcoder.typepad.com/blog/2015/01/getting-the-wall-elevation-profile.html
// Credit to Jeremy Tammik
#!revit24

var uiDoc = uiapp.ActiveUIDocument;
Selection selection = uiDoc.Selection;

Reference picked = selection.PickObject(ObjectType.Element, "Pick a wall");

var selWall = doc.GetElement(picked) as Wall;


// Get the external wall face for the profile
IList<Reference> sideFaces
  = HostObjectUtils.GetSideFaces(selWall,
    ShellLayerType.Exterior );
 
Element e2 = doc.GetElement( sideFaces[0] );

Face face = e2.GetGeometryObjectFromReference(
  sideFaces[0] ) as Face;

IList<CurveLoop> curveLoops
  = face.GetEdgesAsCurveLoops();

// ExporterIFCUtils class can also be used for 
// non-IFC purposes. The SortCurveLoops method 
// sorts curve loops (edge loops) so that the 
// outer loops come first.
IList<IList<CurveLoop>> curveLoopLoop
  = ExporterIFCUtils.SortCurveLoops(
    curveLoops );

List<List<Line>> wallLines = new List<List<Line>>();

foreach(var list in curveLoopLoop){

    foreach(var lines in list){
          var subList = new List<Line>();
          foreach(Curve curve in lines){
            Line line = curve as Line;
            if (line!=null){
                subList.Add(line);
            }
          }
          wallLines.Add(subList);
    }

}

display(curveLoopLoop);
wallLines

.NET interactive will have some trouble serializing a Revit Curve to JSON but the XYZ is fine. Let's build a dto.

In [ ]:
#!revit24
var dto = wallLines.Where(l => l.Count > 0).Select(boundary => boundary.Select(ln => new { start = ln.Origin, end = ln.GetEndPoint(1)}).ToList());
display(dto);
dto

Use the #!set magic command to share the dto with the C#-kernel.

In [ ]:
#!set --value @revit24:dto --name wallLines


Hypar Elements is a cross-platform library for creating building elements. It brings an interactive extension that enables us to show the geometry live in the notebook. 

In [ ]:
#r "nuget: Hypar.Elements, *-*"

In [22]:
var polylines = new List<Polyline>();
foreach (var boundary in wallLines.RootElement.EnumerateArray())
{
   
   var points = new List<Vector3>();

   foreach (var lineJson in boundary.EnumerateArray())
   {
   
      var start = lineJson.GetProperty("start");
      var end = lineJson.GetProperty("end");
      

      var a = new Vector3((double)start.GetProperty("X").GetDouble(),start.GetProperty("Y").GetDouble(),
                           start.GetProperty("Z").GetDouble());
      var b = new Vector3((double)end.GetProperty("X").GetDouble(),end.GetProperty("Y").GetDouble(),end.GetProperty("Z").GetDouble());
      
      points.Add(a);
      points.Add(b);

   }

   Polyline polyline = new Polyline(points);
   polylines.Add(polyline);
}

return polylines;
